In [1]:
import matplotlib.pyplot as plt
import numpy as np
import intake
from dask.diagnostics import ProgressBar

In [2]:
from cmip6_preprocessing.utils import google_cmip_col

# Initialize the Pangeo CMIP6 cloud collection
col = google_cmip_col() 


## Filtering the data using specifications 

In [3]:
def model_list(experiment_id1,experiment_id2,variable_id1,variable_id2):
    list1 = col.df[col.df.experiment_id == experiment_id1].source_id.unique()
    list2 = col.df[col.df.experiment_id == experiment_id2].source_id.unique()

    elist = np.intersect1d(list1,list2)

    # all source_id with variable_id == 'no3os' or 'tos'
    vlist1 = col.df[col.df.variable_id == variable_id1].source_id.unique()
    vlist2 = col.df[col.df.variable_id == variable_id2].source_id.unique()

    vlist = np.intersect1d(vlist1,vlist2)

    model = np.intersect1d(vlist,elist)
    return model

In [4]:
model_list('historical','ssp585','tos','no3os')

array(['ACCESS-ESM1-5', 'CESM2', 'CESM2-WACCM', 'CNRM-ESM2-1', 'CanESM5',
       'CanESM5-CanOE', 'GFDL-ESM4', 'GISS-E2-1-G', 'IPSL-CM6A-LR',
       'MIROC-ES2L', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NorESM2-LM',
       'NorESM2-MM', 'UKESM1-0-LL'], dtype=object)

***This is how we can know that how many different models are there with that perticular specifications***

But this has one issue that few models make their way in the list which have only one `variable_d`

In [5]:
models = model_list('historical','ssp585','tos','no3os')

cat = col.search(
    source_id=models,
    grid_label='gn',
    table_id='Omon',
    member_id = ['r2i1p1f1', 'r3i1p1f1', 'r2i1p2f1', 'r3i1p2f1'] 
)
cat.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id", "member_id"]
].nunique()


,experiment_id,variable_id,table_id,member_id
source_id,,,,
ACCESS-ESM1-5,8,59,1,2
CESM2,2,109,1,2
CESM2-WACCM,5,108,1,2
CanESM5,16,66,1,4
CanESM5-CanOE,7,68,1,2
GFDL-ESM4,3,10,1,2
GISS-E2-1-G,6,29,1,2
IPSL-CM6A-LR,11,109,1,2
MPI-ESM1-2-HR,6,91,1,2


We can see that `GFDL-ESM4` , `GISS-E2-1-G` and `NorESM2-MM` have only one variable_id and this can create problem in our analysis

***One another way is to use `search` method and force it to apply all query***

In [6]:
#these are the models that julius has taken in his notebook

models = ['CanESM5-CanOE', 'IPSL-CM6A-LR']

# Define our query
query = dict(
    variable_id=['tos','no3os'],
    source_id=models,
    experiment_id=['historical', 'ssp585'],
    grid_label='gn',
    table_id='Omon',
    member_id = ['r2i1p1f1', 'r3i1p1f1', 'r2i1p2f1', 'r3i1p2f1'] 
)

cat = col.search(require_all_on=["source_id"], **query)
cat.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id", "member_id"]
].nunique()

/srv/conda/lib/python3.9/site-packages/intake_esm/search.py:104: UserWarning: Query returned zero results.
  warn(message)


,experiment_id,variable_id,table_id,member_id
source_id,,,,


***We can see that it is producing zero result***

***Where as if I comment this member_id it will give me total models. That means No model is satisfying exactly these queries***

In [7]:
query = dict(
    variable_id=["tos", "no3os"],
    experiment_id=["historical", "ssp585"],
    table_id=["Omon"],
    source_id=['CanESM5-CanOE', 'IPSL-CM6A-LR'],
#     member_id = ['r2i1p1f1', 'r3i1p1f1', 'r2i1p2f1', 'r3i1p2f1'],
    grid_label='gn'
)

In [8]:
col_subset = col.search(require_all_on=["source_id"], **query)

In [9]:
col_subset.df.groupby("source_id")[["experiment_id", "variable_id", "table_id", "member_id"]].nunique()


,experiment_id,variable_id,table_id,member_id
source_id,,,,
CanESM5-CanOE,2,2,1,3
IPSL-CM6A-LR,2,2,1,32


## Now going with one method described above and try to analyse it

In [10]:
models = ['ACCESS-ESM1-5','CESM2','CESM2-WACCM']

cat = col.search(
    source_id=models,
    grid_label='gn',
    table_id='Omon',
    member_id = ['r2i1p1f1', 'r3i1p1f1', 'r2i1p2f1', 'r3i1p2f1'] 
)
cat.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id", "member_id"]
].nunique()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [11]:
from cmip6_preprocessing.preprocessing import combined_preprocessing
# load all datasets into a python dictionary
ddict = cat.to_dataset_dict(
    zarr_kwargs={'consolidated':True, 'use_cftime':True}, # recommended for faster reading and better time handling
    storage_options={'token': 'anon'}, # needed to access the public CMIP6 data on google
    aggregate=False,
    preprocess=combined_preprocessing, # this applies the preprocessing to all datasets
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/srv/conda/lib/python3.9/site-packages/cmip6_preprocessing/preprocessing.py:233: UserWarning: ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r3i1p1f1.Omon.gn.v20191203.sos: Unit correction failed with: Cannot parse units:
    invalid units for variable 'sos': 0.001 (attribute) (reason: Unit expression cannot have a scaling factor.)
  warnings.warn(
/srv/conda/lib/python3.9/site-packages/cmip6_preprocessing/preprocessing.py:233: UserWarning: CMIP.NCAR.CESM2-WACCM.historical.r2i1p1f1.Omon.gn.none.sos: Unit correction failed with: Cannot parse units:
    invalid units for variable 'sos': 0.001 (attribute) (reason: Unit expression cannot have a scaling factor.)
  warnings.warn(
/srv/conda/lib/python3.9/site-packages/cmip6_preprocessing/preprocessing.py:233: UserWarning: CMIP.CSIRO.ACCESS-ESM1-5.historical.r3i1p1f1.Omon.gn.v20191203.sos: Unit correction failed with: Cannot parse units:
    invalid units for variable 'sos': 0.001 (attribute) (reason: Unit expression cannot have a scaling factor.)
  

RuntimeError: Failed to apply pre-processing function: combined_preprocessing

In [ ]:
# quick example of how to loop over several models
from cmip6_preprocessing.utils import cmip6_dataset_id
fig, axarr = plt.subplots(ncols=3, nrows=4, figsize=[30,20])
for ax, (name, ds) in zip(axarr.flat, ddict.items()):
    # select the first time step
    ds = ds.isel(time=100)
    # select the datavariable
    da = ds[ds.variable_id]
    # plot
    
    da.plot(ax=ax)
    ax.set_title('.'.join(cmip6_dataset_id(ds).split('.')[2:6]))
    
fig.subplots_adjust(hspace=0.5, wspace=0.5)

In [ ]:
from cmip6_preprocessing.postprocessing import merge_variables, concat_members, concat_experiments
ddict_combined_a = merge_variables(ddict)
def maybe_remove_attrs(ds, attr):
    if attr in ds.attrs.keys():
        del ds.attrs[attr]
    return ds
# remove the `variable_id` attr manually
ddict_combined_aa = {k:maybe_remove_attrs(ds, 'variable_id') for k,ds in ddict_combined_a.items()}
ddict_combined_b = concat_experiments(ddict_combined_aa)
# same thing for experiment_id
ddict_combined_bb = {k:maybe_remove_attrs(ds, 'experiment_id') for k,ds in ddict_combined_b.items()}
ddict_combined = concat_members(ddict_combined_bb)

In [ ]:
ddict_combined.keys()

In [ ]:
ddict_combined['MPI-ESM1-2-LR.gn.Omon']

In [ ]:
ddict_combined['ACCESS-ESM1-5.gn.Omon']

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=[15,5])
for ax, (name, ds) in zip(axarr.flat, ddict_combined.items()):
    # show just the first time step
    ds = ds.isel(time=0)
    
    # mask out the SST where the SST is above 25 deg
    da = ds.tos.where(ds.tos>=25)
    
    # average all members
    if 'member_id' in da.dims:
        da = da.mean('member_id')
    da.plot(ax=ax, robust=True)
    ax.set_title(name)
fig.subplots_adjust(hspace=0.5, wspace=0.5)